# Install and load all libraries

In [1]:
from ml_utils import *
from fastai.vision.all import *
import gc

# Creating differe dataloaders

In [2]:
def train(exp_name,arch,size,item_tfms,epochs=12):
    dblock = get_dblock(item_tfms=item_tfms, batch_tfms=[*aug_transforms(size=size, min_scale=1),Normalize.from_stats(*imagenet_stats)])
    dls = dblock.dataloaders(images_df, seed=42, n_workers=32)
    learn = vision_learner(dls, arch, loss_func=combo_loss, metrics=metrics_cfg, n_out=cfg(4)).to_fp16()
    learn.fine_tune(epochs, 0.01)
    learn.path=Path('models/small')
    learn.export(f'{arch}-model.pkl')
    return learn.tta(dl=dls.valid)
 

In [3]:
res = (180,320)

In [7]:
archs = {
    # 'convnext_small_in22k': {
    #     (Resize(res, ResizeMethod.Squish), res),
    # },
    # 'vit_small_patch16_224': {
    #     (Resize(res, ResizeMethod.Squish), 224),
    # },
    # 'swinv2_base_window12_192_22k':{
    #     (Resize(res, ResizeMethod.Squish), 192),
    # },
    'swin_small_patch4_window7_224': {
        (Resize(res, ResizeMethod.Squish), 224)
    }
}

In [6]:
tta_res = []

for arch, details in archs.items():
    for i, s in details:
        print('----',arch, ' --- ')
        print(s)
        print(i.name)
        tta_res.append(train('squish', arch, size=s, item_tfms=i))
        gc.collect()
        torch.cuda.empty_cache()


---- swinv2_base_window12_192_22k  --- 
192
Resize -- {'size': (320, 180), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


/home/bilal/mambaforge/lib/python3.9/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646756402876/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,0.498197,0.220209,0.056881,0.029022,0.051695,0.082611,0.018138,0.007911,0.015117,0.025654,0.066820,3:15:10


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,0.059888,0.120811,0.024021,0.014216,0.037795,0.044780,0.005354,0.003365,0.007949,0.009859,0.026527,4:02:58
1,0.047528,0.141026,0.020881,0.019087,0.042134,0.058924,0.004730,0.003871,0.008564,0.010528,0.027693,4:03:06
2,0.062711,0.178414,0.032194,0.020391,0.056606,0.069224,0.005406,0.003182,0.008919,0.010340,0.027847,4:02:37
3,0.037430,0.188633,0.035674,0.018384,0.058485,0.076091,0.006454,0.003386,0.008549,0.010308,0.028698,4:02:28


KeyboardInterrupt: 

In [9]:
for arch, details in archs.items():
    for i, s in details:
        print('----',arch, ' --- ')
        print(s)
        print(i.name)
        tta_res.append(train('squish', arch, size=s, item_tfms=i))
        gc.collect()
        torch.cuda.empty_cache()

---- swin_small_patch4_window7_224  --- 
224
Resize -- {'size': (320, 180), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,0.656131,0.324544,0.086730,0.043054,0.069851,0.124909,0.027503,0.012269,0.020436,0.039528,0.099736,2:11:13


epoch,train_loss,valid_loss,usm1_loss,usm2_loss,usm3_loss,usm4_loss,usm1_err,usm2_err,usm3_err,usm4_err,combo_err,time
0,0.095625,0.120441,0.022142,0.014914,0.035180,0.048206,0.005764,0.002934,0.007818,0.011547,0.028063,2:41:36
1,0.062934,0.133132,0.026951,0.015966,0.042626,0.047590,0.006171,0.003500,0.008537,0.009938,0.028147,2:41:14
2,0.052407,0.170987,0.030042,0.020906,0.048181,0.071858,0.005767,0.003214,0.008646,0.010563,0.028190,2:41:24
3,0.066914,0.176061,0.032060,0.020281,0.052581,0.071140,0.005310,0.003477,0.008892,0.009727,0.027406,2:41:19
4,0.033143,0.198849,0.031659,0.022107,0.066409,0.078673,0.004445,0.003056,0.008219,0.008953,0.024673,2:41:14
5,0.031553,0.178320,0.033801,0.019111,0.056508,0.068900,0.005020,0.002870,0.008095,0.008872,0.024858,2:41:03
6,0.022781,0.161876,0.025608,0.020006,0.057031,0.059231,0.004605,0.002837,0.008018,0.007646,0.023105,2:40:58
7,0.018647,0.199529,0.034291,0.021230,0.067651,0.076357,0.004097,0.002956,0.007875,0.008777,0.023705,2:41:05
8,0.010374,0.184743,0.027902,0.019763,0.061701,0.075377,0.003356,0.002306,0.007287,0.007585,0.020535,2:40:49
9,0.007659,0.200087,0.035706,0.020586,0.067213,0.076583,0.003653,0.002187,0.007061,0.007545,0.020446,2:41:23


In [89]:
save_pickle('06_archs-small-ensemble-tta_res.pkl', tta_res)

In [86]:
def decode_ensemble_result(output):
    output = first(zip(*output)) # unzipping preds and targes
    output+= output[1:3]        # increase weights of certain models with higher accuracy
    output = torch.stack(output)# stacking predctions from all models
    labels=[]
    df = pd.DataFrame(dls.valid.items)
    df.drop(['y1_label','y2_label','y3_label','y4_label','valid'], axis=1, inplace=True)
    for i in range(len(dls.c)):
        arm_preds = output[:,:,cfg(i):cfg(i+1)].mean(0);
        arm_idxs = arm_preds.argmax(dim=1)
        arm_vocab = np.array(dls.vocab[i])
        df[f'y{i+1}_label'] = arm_vocab[arm_idxs]
        
    df['labels_pred'] = '['+ df['y1_label']+', '+df['y2_label']+', '+df['y3_label']+', '+df['y4_label']+']'
    df['accurate_tf'] = (df['labels']==df['labels_pred'])
    df.drop(['y1_label','y2_label','y3_label','y4_label'], axis=1, inplace=True)
    return df


In [88]:
output_df = decode_ensemble_result(tta_res)
output_df.to_csv('06_archs-small-ensemble-results.csv', index=False)
output_df['accurate_tf'].value_counts()/len(output_df)*100

True     97.984198
False     2.015802
Name: accurate_tf, dtype: float64

In [11]:
tta_prs = first(zip(*tta_res))

In [22]:
tta_prs+= tta_prs[1:3]

In [36]:
tta_prs_stacked = torch.stack(tta_prs)

In [59]:
pr_usm1 = tta_prs_stacked[:,:,:cfg(1)].mean(0);
idxs1 = pr_usm1.argmax(dim=1)
vocab = np.array(dls.vocab[0])
usm1_lbl = vocab[idxs1]

pr_usm2 = tta_prs_stacked[:,:,cfg(1):cfg(2)].mean(0);
idxs2 = pr_usm2.argmax(dim=1)
vocab = np.array(dls.vocab[1])
usm2_lbl = vocab[idxs2]

pr_usm3 = tta_prs_stacked[:,:,cfg(2):cfg(3)].mean(0);
idxs3 = pr_usm3.argmax(dim=1)
vocab = np.array(dls.vocab[2])
usm3_lbl = vocab[idxs3]

pr_usm4 = tta_prs_stacked[:,:,cfg(3):cfg(4)].mean(0);
idxs4 = pr_usm4.argmax(dim=1)
vocab = np.array(dls.vocab[3])
usm4_lbl = vocab[idxs4]


In [61]:
val_df = pd.DataFrame(dls.valid.items)
val_df.drop(['y1_label','y2_label','y3_label','y4_label','valid'], axis=1, inplace=True)
val_df['labels_pred'] = ['['+usm1_lbl[i]+','+usm2_lbl[i]+','+usm3_lbl[i]+','+usm4_lbl[i]+']' for i in range(len(usm1_lbl))]

val_df.head(100)

,image_id,clip_name,labels,labels_pred
6,data/train_images_tiny/clip_000003/00000.jpg,clip_000003,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
7,data/train_images_tiny/clip_000003/00060.jpg,clip_000003,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
8,data/train_images_tiny/clip_000004/00000.jpg,clip_000004,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
9,data/train_images_tiny/clip_000004/00060.jpg,clip_000004,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
10,data/train_images_tiny/clip_000005/00000.jpg,clip_000005,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
...,...,...,...,...
539,data/train_images_tiny/clip_000269/00000.jpg,clip_000269,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
572,data/train_images_tiny/clip_000286/00060.jpg,clip_000286,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
573,data/train_images_tiny/clip_000286/00000.jpg,clip_000286,"[needle driver, nan, needle driver, cadiere forceps]","[needle driver,nan,needle driver,cadiere forceps]"
576,data/train_images_tiny/clip_000288/00000.jpg,clip_000288,"[bipolar forceps, nan, monopolar curved scissors, cadiere forceps]","[bipolar forceps,nan,monopolar curved scissors,cadiere forceps]"


In [30]:
pr_usm1 = tta_prs_stacked[:,:,:cfg(1)].mean(0);
idxs1 = pr_usm1.argmax(dim=1)
vocab = np.array(dls.vocab[0])
vocab[idxs1]

TypeError: tuple indices must be integers or slices, not tuple

In [29]:
tta_prs[0][:,cfg(1):cfg(2)].shape

torch.Size([9872, 15])